In [4]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# load dataset and bring it in format for toast
network = RoadNetwork()
network.load("../osm_data/porto")
df = pd.read_csv("../datasets/trajectories/Porto/road_segment_map_final.csv", sep=";", usecols=["id", "cpath"])

In [3]:
print(network.gdf_nodes.shape)
#dG = nx.DiGraph(network.G)
dG = nx.line_graph(network.G, create_using=nx.DiGraph)
index = np.arange(2, len(list(dG.nodes))+2) # index shifted, since 0 and 1 are reserved for mask pad tokens
nx.set_node_attributes(dG, dict(zip(list(dG.nodes), index)), name="index")


(5358, 6)


In [51]:
nodes = list(dG.nodes(data=True))

for n, d in nodes:
    print(network.gdf_edges.loc[n, :]["highway"])
    break


motorway_link
